In this notebook, we produce the results related to the diffRBM model 
of TCR specificity (Figs. 5,6)

In [ ]:
### Inclusions ###

rootf = ... ## Your folder ##
# put here the folder where all subfolders (diffRBM, Align utils, TCR_specificity_model etc. are saved)

import sys, os, pickle
sys.path.append('/home/barbara/.local/lib/python3.9/site-packages/')
sys.path.append(rootf)
sys.path.append(rootf + '/diffRBM/source/')
sys.path.append(rootf + '/diffRBM/utilities/')

import diffrbm
import csv
import utilities as utilities
import rbm as rbm
import diffrbm as diffrbm

import dataset_utils, RBM_utils, evaluate_learning_utils
import utilities, sequence_logo,plots_utils
import Proteins_utils

import numpy as np
import pandas as pd
import importlib
import utilities_diffrbm

# Plots stuff
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib as mpl
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib import patches
from pandas.plotting import table
mpl.rcParams['font.family'] = ['Garuda']
mpl.rcParams['font.serif'] = ['Garuda-Oblique']

import random
import sklearn
from sklearn import metrics

import torch
import torch.nn
import torch.nn.functional as F
import torch.optim as optim
from scipy.optimize import fsolve
import math

## A series of useful functions ##

def overlap_seqs(list1,list2):
    overlap=[]
    for i in range(len(list1)):
        if list1[i] in list2:
            overlap.append(list1[i])
    return overlap

def align_seqs(all_cdr3,SA,SAmin,SAmax):
    import subprocess, os
    name_mat = rootf + '/Align_utils/align_protpy.py'
    all_seqs1 = list(all_cdr3)
    name_seed = rootf + '/Align_utils/prots_seed.txt'
    with open(name_seed, 'w') as out_f:
        for u in range(len(all_seqs1)):
            if len(all_seqs1[u]) >= SAmin:  
                out_f.write(all_seqs1[u] + '\n')
                
    subprocess.call('python3 ' + name_mat + ' -ss ' + name_seed + ' -SA ' + str(SA) + ' -SAmin ' + str(SAmin) + ' -SAmax ' + str(SAmax), shell = True)
    seqs_al = np.loadtxt(rootf + '/Align_utils/aligned_prot.txt')
    os.system('rm ' + rootf + '/Align_utils/aligned_prot.txt')
    
    return seqs_al

def align_seqs_seed(seed_seqs,new_seqs,SA,SAmin,SAmax):
    
    import subprocess, os
    name_mat = rootf + '/Align_utils/align_prot_to_seedpy.py'
    
    all_seqs1 = list(seed_seqs)
    name_seed = rootf + '/Align_utils/prots_seed.txt'
    with open(name_seed, 'w') as out_f:
        for u in range(len(all_seqs1)):
            if len(all_seqs1[u]) >= SAmin:  
                out_f.write(all_seqs1[u] + '\n')
                
    seqs_gapless = list(new_seqs)
    name_seqs = rootf + '/Align_utils/prots_seqs.txt'
    with open(name_seqs, 'w') as out_f:
        for u in range(len(seqs_gapless)):
            if len(seqs_gapless[u]) >= SAmin:  
                out_f.write(seqs_gapless[u] + '\n')
                
    subprocess.call('python3 ' + name_mat + ' -sseed ' + name_seed + ' -sseqs ' + name_seqs + ' -SA ' + str(SA) + ' -SAmin ' + str(SAmin) + ' -SAmax ' + str(SAmax) + ' -yw 0', shell = True)
    seqs_al = np.loadtxt(rootf + '/Align_utils/aligned_prot.txt')
    os.system('rm ' + rootf + '/Align_utils/aligned_prot.txt')
    
    return seqs_al

def flatten_list(listoflist):
    listoflist_fl = [];
    for l in range(len(listoflist)):
        for u in range(len(listoflist[l])):
            listoflist_fl.append(listoflist[l][u])
    return listoflist_fl
 
curr_int = np.int16
def convert_number(seqs): # convert to numbers already aligned seqs
    aa = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V',  'W', 'Y','-']
    aadict = {aa[k]: k for k in range(len(aa))} 
    
    msa_num = np.array(list(map(lambda x: [aadict[y] for y in x], seqs[0:])), dtype=curr_int, order="c") ### Here change ####
    
    return msa_num

def uniqueIndexes(l):
    seen = set()
    res = []
    for i, n in enumerate(l):
        if n not in seen:
            res.append(i)
            seen.add(n)
    return res

def convert_letter(seqs_n): # convert to numbers already aligned seqs
    aa = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V',  'W', 'Y','-']
    aadictinv = {k: aa[k] for k in range(len(aa))} 
    seqs=[]
    if type(seqs_n[0]) == curr_int:
        seqs.append(''.join([aadictinv[e] for e in seqs_n]))
    else:
        for t in range(len(seqs_n)):
            seqs.append(''.join([aadictinv[e] for e in seqs_n[t]]))
    return seqs

def produce_callback_weights(RBMdiff):
    def callback():
        RBMdiff.RBMpost.weights[RBMdiff.n_h_:] = 0
    return callback

def loglikelihood_indip_model(fields, logZ, seqs):
    return fields[np.arange(len(fields)), seqs].sum(axis=1) - logZ

def add_pseudocount(fields, n):
    return np.array([(f + 1/n) / np.sum((f + 1/n)) for f in fields])

def regularized_pwm(emp_freqsF, l2, pc):
    
    fields=[]
    pwmi=[]
    for ff in range(len(emp_freqsF)):
        def equations(x):
            return [(math.exp(x[i])/sum([math.exp(x[i]) for i in range(len(x))]) + l2*x[i] - emp_freqs[i]) for i in range(len(x))]
        emp_freqs = np.copy(emp_freqsF[ff])
        init = [np.log(c + pc) for c in emp_freqs]
        fields0 = fsolve(equations, init)
        pwmi0 = [math.exp(fields0[j])/sum([math.exp(fields0[i]) for i in range(len(fields0))]) for j in range(len(fields0))]
        
        fields.append(list(fields0))
        pwmi.append(pwmi0)
        
    fields=np.array(fields)
    pwmi=np.array(pwmi)
        
    return (fields, pwmi)

def regularized_diffpwm(emp_freqsF, l2, pc, fields_backF):
    
    fields=[]
    pwmi=[]
    for ff in range(len(emp_freqsF)):
        emp_freqs = np.copy(emp_freqsF[ff])
        fields_back = np.copy(fields_backF[ff])
        
        def equations(x):
            return [(math.exp(x[i] + fields_back[i])/sum([math.exp(x[i] + fields_back[i]) for i in range(len(x))]) + l2*(x[i]+ fields_back[i]) - emp_freqs[i]) for i in range(len(x))]
        
        
        init = [np.log(c + pc) for c in emp_freqs]
        fields0 = fsolve(equations, init)
        pwmi0 = [math.exp(fields0[j] + fields_back[j])/sum([math.exp(fields0[i] + fields_back[i]) for i in range(len(fields0))]) for j in range(len(fields0))]
        
        fields.append(list(fields0))
        pwmi.append(pwmi0)
        
    fields_top = np.array(fields)
    pwmi_top = np.array(pwmi)
        
    return (fields_top, pwmi_top)

import copy
def site_conservation_scores(RBM, seq, A=20, L=9):
    F = np.zeros((L, A))
    for i in range(L):
        for a in range(A):
            v = copy.copy(seq)
            v[i] = a
            F[i,a] = RBM.free_energy(v)
    return F.mean(1) - RBM.free_energy(v)

def gene_to_num_str(gene_name, gene_type):
    """Strips excess gene name info to number string.

    Parameters
    ----------
    gene_name : str
        Gene or allele name
    gene_type : char
        Genomic cassette type. (i.e. V, D, or J)
    Returns
    -------
    num_str : str
        Reduced gene or allele name with leading zeros and excess
        characters removed.

    """
    # get rid of allele
    gene_name=gene_name.split('*')[0]
    num_str = gene_type.lower().join([g.lstrip('0') for g in gene_name.lower().split(gene_type.lower())[1:]])
    num_str = '-'.join([g.lstrip('0') for g in num_str.split('-')])
    return gene_type.lower() + num_str.replace('/', '')

def shorten_gene_name(gene):
    """Removes 0 in gene name when not needed."""
    pre = gene[:4]
    first_num = str(int(gene[4:6]))
    if len(gene[6:]) == 0:
        short_gene = pre + first_num
    else:
        dash = gene[6:7]
        second_num = str(int(gene[7:]))
        short_gene = pre + first_num + dash + second_num
    return short_gene

def add_VJ_info_2num(seqs_2num, V_list, J_list, V_dict, J_dict):
    Vlist_2num = np.array([V_dict[v] for v in V_list]).reshape((seqs_2num.shape[0], 1))
    Jlist_2num = np.array([J_dict[j] for j in J_list]).reshape((seqs_2num.shape[0], 1))
    seqs_2num_VJ = np.append(seqs_2num, Vlist_2num, axis=1)
    seqs_2num_VJ = np.append(seqs_2num_VJ, Jlist_2num, axis=1)
    return seqs_2num_VJ


def extract_data(pepN):

    dataset = 'diffRBM_'+ pepN ## This is the name of the folder with the model; if does not exist I create it 
    name_folder = rootf + '/TCR_specificity_model/' + dataset
    # load JV2num dicts
    t_df = pd.read_csv(name_folder + '/J2num_dict_train_train.csv')
    J2num_dict = dict(zip(t_df['J_gene'], t_df['RBM_VU_id']))
    t_df = pd.read_csv(name_folder + '/V2num_dict_train_train.csv')
    V2num_dict = dict(zip(t_df['V_gene'], t_df['RBM_VU_id']))
    vdgdb_df = pd.read_csv(rootf + '/TCR_specificity_model/diffRBM_' + pepN + '/VDJdb_' + pepN + '_WithAligned20.csv')
    train_datan = vdgdb_df.drop_duplicates().reset_index(drop=True)
    # do num2seq
    t_trainseq = train_datan["ali_seq"].to_list()
    trainseq_2num = np.array(Proteins_utils.seq2num(t_trainseq), dtype=np.int16)
    # add to gen_seqs2num two integers for each row, obtained from V and J
    Vlist = train_datan["TRBV_gene"].to_list()
    Jlist = train_datan["TRBJ_gene"].to_list()
    train_2num_withVJ = add_VJ_info_2num(trainseq_2num, Vlist, Jlist, V2num_dict, J2num_dict)
    if makeVJ:
        train_dataBIvn = np.copy(train_2num_withVJ)
    else:
        train_dataBIvn = np.copy(train_2num_withVJ[:,:-2])

    return train_dataBIvn

def regularized_pwm_torch(freqs, l2, n_iter = 1000):
    g = torch.zeros_like(freqs, requires_grad=True)
    opt = torch.optim.LBFGS([g])
    def ll(g):
        return torch.sum(freqs * g) - torch.sum(torch.logsumexp(g, 1)) - l2/2 * torch.norm(g)**2
    for t in range(n_iter):
        opt.zero_grad()
        loss = -ll(g)
        loss.backward()
        opt.step(lambda: -ll(g))
        opt.zero_grad()
        
    N = freqs.shape[0]
    g_pwm  = g.exp() / g.exp().sum(dim=1).reshape(N,1) + l2 * g
    
    fields = g.detach().numpy()
    pwmi = g_pwm.detach().numpy()
    
    return (fields, pwmi)

Here we train and test (by the AUC of discrimination of specific vs background receptors) the diffRBM TCR specificity models and compare them to other RBM and PWM-based approaches

In [ ]:
## These are combinations of options: whether to include VJ segments, what background dataset to use
combinations = [[1,'emerson_training_test'], [0,'emerson_training_test'], [1,'dean_training_test']]

for combo in combinations:
    
    makeVJ = combo[0] ## just to decide if to include V and J or not ##

    if makeVJ:
        strVJ = 'withVJ'
    else:
        strVJ = 'withoutVJ'
        
    replBB = 0
    add_vj = strVJ

    list_pepa=['YLQPRTFLL','NLVPMVATV','GLCTLVAML','GILGFVFTL']
    list_pep=list_pepa
    
    dataset_back = combo[1]
    if dataset_back == 'emerson_training_test':
        add_str=''
    else:
        add_str='D'

    for pep in list_pep:

        SA=20

        dataset = 'diffRBM_'+ pep ## This is the name of the folder with the model; if does not exist I create it 
        name_folder = rootf + '/TCR_specificity_model/' + dataset # Name folder reflects what peptide-specific dataset one is considering #
       
        if os.path.exists(name_folder) is False:
            os.mkdir(name_folder)

        name_mf = '/models'
        if os.path.exists(name_folder + name_mf + '/') is False:
            os.mkdir(name_folder + name_mf + '/')

        name_mff = '/models/indices'
        if os.path.exists(name_folder + name_mff + '/') is False:
            os.mkdir(name_folder + name_mff + '/')

        # load JV2num dicts - see comment below
        t_df = pd.read_csv(name_folder + '/J2num_dict_train_train.csv')
        J2num_dict = dict(zip(t_df['J_gene'], t_df['RBM_VU_id']))
        t_df = pd.read_csv(name_folder + '/V2num_dict_train_train.csv')
        V2num_dict = dict(zip(t_df['V_gene'], t_df['RBM_VU_id']))

        vdgdb_df = pd.read_csv(rootf + '/TCR_specificity_model/diffRBM_' + pep + '/VDJdb_' + pep + '_WithAligned20.csv')
        train_data = vdgdb_df.drop_duplicates().reset_index(drop=True)
        # do num2seq
        t_trainseq = train_data["ali_seq"].to_list()
        trainseq_2num = np.array(Proteins_utils.seq2num(t_trainseq), dtype=np.int16)
        # add to gen_seqs2num two integers for each row, obtained from V and J
        Vlist = train_data["TRBV_gene"].to_list()
        Jlist = train_data["TRBJ_gene"].to_list()

        train_2num_withVJ = add_VJ_info_2num(trainseq_2num, Vlist, Jlist, V2num_dict, J2num_dict)

        if makeVJ:
            train_dataI = np.copy(train_2num_withVJ)
        else:
            train_dataI = np.copy(train_2num_withVJ[:,:-2])

        Mb=1000000
        filename_cdr3 = rootf + '/TCR_specificity_model/' + dataset_back + '/train_data_0_aligned_20.txt' 
        filename_cdr3raw = rootf + '/TCR_specificity_model/' + dataset_back + '/train_data_0.txt' 
        train_seq0=[]
        with open(filename_cdr3) as f:
            for line in f:
                linesplit = line.strip().split('\n')
                train_seq0.append(linesplit[0])
        train_dataB_num = convert_number(train_seq0)                       
        if makeVJ:
            ffraw = pd.read_csv(filename_cdr3raw,sep='\t',header=None)
            VlistB = list(np.array(ffraw)[:,1])
            JlistB = list(np.array(ffraw)[:,2])
            train_2num_withVJ_B = add_VJ_info_2num(train_dataB_num, VlistB, JlistB, V2num_dict, J2num_dict)
            train_dataBI = train_2num_withVJ_B[:Mb]
        else:
            train_dataBI = train_dataB_num[:Mb]

        seqs_temp = list(train_dataI)
            
        Mbv=10000
        filename_cdr3 = rootf + '/TCR_specificity_model/'+dataset_back+'/train_data_1_aligned_20.txt' 
        filename_cdr3raw = rootf + '/TCR_specificity_model/'+dataset_back +'/train_data_1.txt' 
        inds_non_overlap = np.loadtxt(rootf + '/TCR_specificity_model/' + dataset_back + '/1_inds_nonoverlap_0.txt').astype(np.int16)
        train_seq0=[]
        with open(filename_cdr3) as f:
            for line in f:
                linesplit = line.strip().split('\n')
                train_seq0.append(linesplit[0])
        train_dataB_num = convert_number(train_seq0)                       
        if makeVJ:
            ffraw = pd.read_csv(filename_cdr3raw,sep='\t',header=None)
            VlistB = list(np.array(ffraw)[:,1])
            JlistB = list(np.array(ffraw)[:,2])
            train_2num_withVJ_B = add_VJ_info_2num(train_dataB_num, VlistB, JlistB, V2num_dict, J2num_dict)
            train_dataBIv = train_2num_withVJ_B[inds_non_overlap[:Mbv]]
        else:
            train_dataBIv = train_dataB_num[inds_non_overlap[:Mbv]]

        train_dataBIv0 = np.copy(train_dataBIv)
        
        first_run = 0
        makefig=0
        maketrainingpwm=0
        make_pos=1
        maketrainingP=0
        maketraining=0
        maketrainingB=0

        print_table = 0
        makeRW=0
        maketrainingN=0
        maketrainingpwmN=0
        list_negs=['control']

        for name_neg in list_negs:
           
            if makeVJ:
                n_cv = np.max(np.unique(train_dataBI[:,-2])) + 1
            else:
                n_cv=21

            seqs_tempN = list(train_dataBIv)
            
            listreg = [0.01]
            listhu = [20] ## lists of hidden units in top RBM
            listregB = [0.001] ## lists of weight regularization in top RBM
            listhuB = [100]## lists of hidden units in top RBM

            ## Various parameters ##
            th = 0.15 ## similarity threshold if you set the reweighting
            vv = 1
            what_l12='Standard'
            what_l2='Standard'
            
            ## RBM parameters ##
            ZF = False ## control the introduction of fields ##
            BN = True ## control batch_size ##

            visible = 'Potts' # Nature of visible units potential. Here, Potts states.
            hidden = 'dReLU' # Nature of hidden units potential. Here, dReLU potential.
            
            seed = 0
            decay_after = 0.5 # Decay learning rate after 50% of iterations (default: 0.5). Value for RBM shown in paper: 0.5
            N_MC = 15

            RR=50
            npw=250

            ## percentage to use in the training dataset ##
            B = 80 ## percentage for positives ##
            Bm = 80
            BB = 100 ## percentage for background ##

            if not(first_run) and maketraining:
                print("ATTENTION YOU ARE RETRAINING THE MODELS")
            if first_run and not(maketraining):
                print("ATTENTION YOU ARE NOT TRAINING THE MODELS")

            log_file = name_folder + name_mf + '/Log.txt'
            print('Start training for peptide ' + pep + '\n')
            f = open(log_file,'a+')
            f.write('Start training for peptide ' + pep + '\n')
            f.close()

            for lib in listreg:
                for hh in listhu:
                    for hhB in listhuB:
                        for libB in listregB:

                            ## DIFFRBM and RBM ##
                            ## disinguish positives from negatives and vice-versa ##
                            list_auc_diff=[] ## diffRBM
                            list_auc_top=[] ## top part of diffRBM
                            list_auc_diffN=[] ## diffRBM (negatives) 
                            list_auc_topN=[] ## top part of diffRBM (negatives)
                            list_auc_difftops=[] ## difference of tops
                            list_auc_diffdiff=[] ## difference of diff (same as top)
                            list_auc_rbm=[] # normal RBM
                            list_auc_diffrbm=[] # difference of normal RBMs
                            
                            # DIFFRBM with Linear top #
                            list_auc_diff_lin=[]
                            list_auc_diff_linN = []
                            list_auc_top_lin = []
                            list_auc_top_linN = []
                            list_auc_difftops_lin=[]

                            ## Alternative approaches: independent-site models ## 
                            list_auc_pwm=[]
                            list_auc_diffpwm=[]

                            ## Alternative approaches: differential independent-site models ## 
                            list_auc_top_dpwm_pos=[]
                            list_auc_diff_dpwm_pos=[]
                            list_auc_top_dpwm_neg=[]
                            list_auc_diff_dpwm_neg=[]
                            list_auc_top_dpwm_diff=[]

                            ## discrimination performance of the background
                            list_auc_backrbm=[]
                            list_auc_backpwm=[]

                            for repl in range(RR):
    
                                ## Reshuffle data before dividing - I also save reshuffled indices ##

                                path_o = name_folder + name_mff + '/index_permutation_repl' + str(repl) + '.txt'
                        
                                if first_run:
                                    full_int = list(np.arange(int(len(seqs_temp)*1)))
                                    full_intR = random.sample(full_int, int(1*(len(seqs_temp))))
                                    np.savetxt(path_o, np.array(full_intR).astype(int), fmt="%i")

                                full_intR = list((np.loadtxt(path_o)).astype(np.int))
                                full_intRp = list((np.loadtxt(path_o)).astype(np.int))
                                data = np.array([seqs_temp[t] for t in full_intR])

                                path_o = name_folder + name_mff + '/index_permutationN_repl' + str(repl) + '.txt'

                                if first_run:
                                    full_int = list(np.arange(int(len(seqs_tempN)*1)))
                                    full_intR = random.sample(full_int, int(1*(len(seqs_tempN))))
                                    np.savetxt(path_o, np.array(full_intR).astype(int), fmt="%i")

                                full_intR = list((np.loadtxt(path_o)).astype(np.int))

                                full_intRn = list((np.loadtxt(path_o)).astype(np.int))
                                train_dataBIv = np.array([seqs_tempN[t] for t in full_intR])

                                train_data = data[:int(B*len(data)/100)]
                                val_data = data[int(B*len(data)/100):]
                                #train_data_all.append(data)
                                
                                if makeRW:
                                    reweight = utilities_diffrbm.compute_MSA_weights(train_data, threshold = th)
                                Mt = len(train_data)

                                train_dataB = train_dataBI[:int(BB*len(train_dataBI)/100)]
                                val_dataB = train_dataBIv[:len(val_data)]
                                
                                val_dataN = np.copy(val_dataB)
                                MtN = len(train_dataB)

                                Mb = len(train_dataB)

                                if makeRW:
                                    reweightB = utilities_diffrbm.compute_MSA_weights(train_dataB, threshold = th)

                                f = open(log_file,'a+')
                                f.write('Size background ' + str(len(train_dataB)) + '\n')
                                f.close
                                
                                if makefig:
                                    name_w = name_folder + '/presentation/model' + mod + '_weights.png'
                                    weB = RBM_back.weights
                                    interesting_features = list(np.arange(n_hB))
                                    fig = sequence_logo.Sequence_logo_multiple(weB[interesting_features], figsize=(5.5,1.8), ylabel = 'weights ',  ticks_every=5, ticks_labels_size=s2-2) 
                                    

                                if what_l2=='Standard':
                                    reg_fields_diffpos = 1/Mt
                                    reg_fields_diffneg = 1/MtN
                                else:
                                    reg_fields_diffpos = (abs(Mt - Mb) + 1)/Mt
                                    reg_fields_diffneg = (abs(MtN - Mb) + 1)/MtN

                                if what_l12=='Standard':
                                    reg_weights_diffpos = lib
                                    reg_weights_diffneg = lib
                                else:
                                    reg_weights_diffpos = Mb/Mt*lib
                                    reg_weights_diffneg = Mb/MtN*lib

                                n_v = train_dataB.shape[1] # Number of visible units = # sites in alignment.

                                l2f = 1/Mb

                                n_hB = hhB
                                l1bB = libB
                                decay_after = 0.5
                                N_MC = 10

                                batch_size = 2000
                                n_iter = 40
                                replB = replBB

                                name_back = 'backRBM-100hu-lowreg-'+ strVJ +'-train_train_WithoutDuplicates_repl'+str(replB)+'_' + str(Mb) + add_str 
                                nameB = rootf + '/TCR_specificity_model/' + name_back +'.data'

                                if maketrainingB:
                                    RBM_back = rbm.RBM(n_h = hhB, n_v = len(train_dataB[0]),n_cv= n_cv, visible='Potts', hidden='dReLU',random_state = seed, zero_field = False)
                                    if maketrainingB:
                                        RBM_back.fit(train_dataB, weights = None, batch_size = batch_size, n_iter = n_iter, l1b = libB, l2_fields = l2f, N_MC = N_MC, decay_after = decay_after, verbose = 1, shuffle_data=False, CD=False)
                                   
                                        RBM_utils.saveRBM(nameB, RBM_back)

                                RBM_back =  RBM_utils.loadRBM(nameB)
                                
                                l2f = reg_fields_diffpos
                               
                                learning_rate = None # default behaviour
                                decay_after = 0.5
                                N_MC = 10
                               
                                if pep == 'YLQPRTFLL':
                                    bb = 16
                                else:
                                    bb = 100
                                if pep == 'NLVPMVATV': 
                                    bb = 200

                                nni = int(2e4) // (train_data.shape[0] // bb)

                                batch_size = bb
                                n_iter = nni
                                
                                n_v = train_data.shape[1] # Number of visible units = # sites in alignment.
                                name_top = name_folder + name_mf +  '/topmodel_imm_repl' + str(repl) + add_str 
                                n_h_top = hh
                                out_par0 = '_RW' + str(makeRW) + '_TR' + str(B) + add_vj
                                out_par = '_nh' + str(n_h_top) + '_l12' + str(lib) + '_ZF' + str(ZF) + out_par0

                                RBMpost = rbm.RBM(n_h = RBM_back.n_h + n_h_top, n_v = RBM_back.n_v, n_cv= RBM_back.n_cv, visible='Potts', hidden=hidden, zero_field=ZF)
                                dRBM = diffrbm.DiffRBM(RBM_back, RBMpost)
                                dRBM.update_post_from_back(vlayer=True, hlayer=True)
                                
                                if maketraining:
                                    if makeRW:
                                        dRBM.fit_top(train_data, weights=reweight, n_iter=n_iter, batch_size=batch_size, l2_fields = l2f, l1b=reg_weights_diffpos, N_MC=N_MC, decay_after=decay_after, verbose=vv, vverbose=vv, batch_norm=BN)
                                    else:
                                        dRBM.fit_top(train_data, weights=None, n_iter=n_iter, batch_size=batch_size, l2_fields = l2f, l1b=reg_weights_diffpos, N_MC=N_MC, decay_after=decay_after, verbose=vv, vverbose=vv, batch_norm=BN)
                                    RBM_utils.saveRBM(name_top + out_par + '.data', dRBM)
                                
                                dRBM =  RBM_utils.loadRBM(name_top + out_par + '.data')
                                llp = -dRBM.top_rbm().free_energy(val_data.astype(np.int16))
                                lln = -dRBM.top_rbm().free_energy(val_dataN.astype(np.int16))

                                labels = np.hstack((np.ones((len(val_data))), np.zeros((len(val_dataN))) )) 
                                scores = np.hstack((llp, lln))
                                fpr, tpr, thresholds = metrics.roc_curve(labels, scores)
                                nn = metrics.auc(fpr, tpr)
                                list_auc_top.append(nn)

                                llp = -RBM_back.free_energy(val_data.astype(np.int16))
                                lln = -RBM_back.free_energy(val_dataN.astype(np.int16))

                                labels = np.hstack((np.ones((len(val_data))), np.zeros((len(val_dataN))) )) 
                                scores = np.hstack((llp, lln))
                                fpr, tpr, thresholds = metrics.roc_curve(labels, scores)
                                nn = metrics.auc(fpr, tpr)
                                list_auc_backrbm.append(nn)
                                
                                llp = -dRBM.RBMpost.free_energy(val_data.astype(np.int16)) 
                                lln = -dRBM.RBMpost.free_energy(val_dataN.astype(np.int16)) 

                                labels = np.hstack((np.ones((len(val_data))), np.zeros((len(val_dataN))) )) 
                                scores = np.hstack((llp, lln))
                                fpr, tpr, thresholds = metrics.roc_curve(labels, scores)
                                nn = metrics.auc(fpr, tpr)
                                list_auc_diff.append(nn)

                                name_rbm = name_folder + name_mf + '/normmodel_imm_repl' + str(repl) + add_str
                                
                                l2f = 1/Mt
                                n_h = hh

                                batch_size = bb
                                n_iter = nni
                                if make_pos:
                                    RBM = rbm.RBM(visible = visible, hidden = hidden, n_v = n_v, n_h = n_h, n_cv = n_cv, random_state = seed, zero_field=ZF)
                                    if maketrainingP:
                                        if makeRW:
                                            RBM.fit(train_data, weights=reweight, batch_size=batch_size, l2_fields = l2f, n_iter=n_iter, l1b=lib, N_MC=N_MC, decay_after=decay_after, verbose=vv, vverbose=vv)
                                        else:
                                            RBM.fit(train_data, weights=None, batch_size=batch_size, l2_fields = l2f, n_iter=n_iter, l1b=lib, N_MC=N_MC, decay_after=decay_after, verbose=vv, vverbose=vv)
                                        RBM_utils.saveRBM(name_rbm + out_par + '.data', RBM)
                                    RBM =  RBM_utils.loadRBM(name_rbm + out_par + '.data')

                                name_top = name_folder + name_mf + '/topmodellin_imm_repl' + str(repl)
                                n_h_top = hh

                                l2f = reg_fields_diffpos

                                batch_size = bb
                                n_iter = nni

                                RBMpost_lin = rbm.RBM(n_h = RBM_back.n_h + n_h_top, n_v = RBM_back.n_v, n_cv= RBM_back.n_cv, visible=visible, hidden=hidden, zero_field=ZF)
                                dRBM_lin = diffrbm.DiffRBM(RBM_back, RBMpost_lin)
                                dRBM_lin.update_post_from_back(vlayer=True, hlayer=True)

                                if maketraining:
                                    if makeRW:
                                        dRBM_lin.fit_top(train_data, weights=reweight, n_iter=n_iter, batch_size=batch_size, l2_fields = l2f, l1b=reg_weights_diffpos, N_MC=N_MC, decay_after=decay_after, verbose=vv, vverbose=vv, batch_norm=BN, callback = produce_callback_weights(dRBM_lin))
                                    else:
                                        dRBM_lin.fit_top(train_data, weights=None, n_iter=n_iter, batch_size=batch_size, l2_fields = l2f, l1b=reg_weights_diffpos, N_MC=N_MC, decay_after=decay_after, verbose=vv, vverbose=vv, batch_norm=BN, callback = produce_callback_weights(dRBM_lin))
                                    RBM_utils.saveRBM(name_top + out_par + '.data', dRBM_lin)
                                dRBM_lin =  RBM_utils.loadRBM(name_top + out_par + '.data')

        
                                if make_pos:
                                    llp = -RBM.free_energy(val_data.astype(np.int16))
                                    lln = -RBM.free_energy(val_dataN.astype(np.int16))

                                    labels = np.hstack((np.ones((len(val_data))), np.zeros((len(val_dataN))) )) 
                                    scores = np.hstack((llp, lln))
                                    fpr, tpr, thresholds = metrics.roc_curve(labels, scores)
                                    nn = metrics.auc(fpr, tpr)
                                    list_auc_rbm.append(nn)

                                llp = -dRBM_lin.RBMpost.free_energy(val_data.astype(np.int16))
                                lln = -dRBM_lin.RBMpost.free_energy(val_dataN.astype(np.int16))

                                labels = np.hstack((np.ones((len(val_data))), np.zeros((len(val_dataN))) )) 
                                scores = np.hstack((llp, lln))
                                fpr, tpr, thresholds = metrics.roc_curve(labels, scores)
                                nn = metrics.auc(fpr, tpr)
                                list_auc_diff_lin.append(nn)

                                llp = -dRBM_lin.top_rbm().free_energy(val_data.astype(np.int16))
                                lln = -dRBM_lin.top_rbm().free_energy(val_dataN.astype(np.int16))   

                                labels = np.hstack((np.ones((len(val_data))), np.zeros((len(val_dataN))) )) 
                                scores = np.hstack((llp, lln))
                                fpr, tpr, thresholds = metrics.roc_curve(labels, scores)
                                nn = metrics.auc(fpr, tpr)
                                list_auc_top_lin.append(nn)

                                print('I start to compare to a differential PWM')

                                if maketrainingpwm:
                                    if makeRW:
                                        pre_fields = utilities.average(train_dataB, c=n_cv, weights=reweightB)
                                    else:
                                        pre_fields = utilities.average(train_dataB, c=n_cv)

                                    fields_back, pwmB = regularized_pwm_torch(torch.from_numpy(pre_fields), 1/(Mb), n_iter= npw)

                                
                                    if makeRW:
                                        pre_fields = utilities.average(train_data, c=n_cv, weights=reweight)
                                    else:
                                        pre_fields = utilities.average(train_data, c=n_cv)

                                    fields_diff_pos, pwm_diff_pos = regularized_pwm_torch(torch.from_numpy(pre_fields), reg_fields_diffpos, n_iter= npw)

                                    np.savetxt(name_folder + name_mf + '/fields_pos_repl' + str(repl) + add_str + out_par0 +'.txt', fields_diff_pos)
                                    np.savetxt(name_folder + name_mf + '/fields_back' + add_str + out_par0 + '.txt', fields_back)
                                else:
                                    fields_diff_pos = np.loadtxt(name_folder + name_mf + '/fields_pos_repl' + str(repl) + add_str+ out_par0 +'.txt')
                                    fields_back = np.loadtxt(name_folder + name_mf + '/fields_back' + add_str+ out_par0 +'.txt')       

                                fields_top_pos = fields_diff_pos - fields_back
                                
                                pos_v_2num = np.copy(val_data)
                                neg_v_2num = np.copy(val_dataN)
                                # compute AUCs - top_dpwm_pos (trained on positives) - I set logZ=1 since it is useless here
                                scores_positives_val = loglikelihood_indip_model(fields_top_pos, 1, pos_v_2num)
                                scores_negatives_val = loglikelihood_indip_model(fields_top_pos, 1, neg_v_2num)
                                labels = np.hstack((np.zeros((len(scores_negatives_val))), np.ones((len(scores_positives_val))))) 
                                scores = np.hstack((scores_negatives_val, scores_positives_val))
                                fpr, tpr, thresholds = metrics.roc_curve(labels, scores)
                                list_auc_top_dpwm_pos.append(metrics.auc(fpr, tpr))

                                # compute AUCs - diff_dpwm_pos (trained on positives)
                                scores_positives_val = loglikelihood_indip_model(fields_diff_pos, 1, pos_v_2num)
                                scores_negatives_val = loglikelihood_indip_model(fields_diff_pos, 1, neg_v_2num)
                                labels = np.hstack((np.zeros((len(scores_negatives_val))), np.ones((len(scores_positives_val))))) 
                                scores = np.hstack((scores_negatives_val, scores_positives_val))
                                fpr, tpr, thresholds = metrics.roc_curve(labels, scores)    
                                list_auc_diff_dpwm_pos.append(metrics.auc(fpr, tpr))

                                scores_positives_val = loglikelihood_indip_model(fields_back, 1, pos_v_2num)
                                scores_negatives_val = loglikelihood_indip_model(fields_back, 1, neg_v_2num)
                                labels = np.hstack((np.zeros((len(scores_negatives_val))), np.ones((len(scores_positives_val))))) 
                                scores = np.hstack((scores_negatives_val, scores_positives_val))
                                fpr, tpr, thresholds = metrics.roc_curve(labels, scores)    
                                list_auc_backpwm.append(metrics.auc(fpr, tpr))

                    ## Final predictions ##
                    list_aucs1 = [list_auc_top,list_auc_diff, list_auc_top_lin, list_auc_diff_lin, list_auc_top_dpwm_pos,list_auc_diff_dpwm_pos, list_auc_rbm, list_auc_backrbm, list_auc_backpwm]
                    list_aucs_names1 = ['top RBM','diffRBM', 'top RBM (lin)','diffRBM (lin)','top PWM', 'diffPWM', 'Normal RBM', 'RBMback', 'PWMback']

                    fig,ax=plt.subplots()
                    fig.set_figheight(5)
                    fig.set_figwidth(15)

                    meth = list_aucs_names1
                    meth_val = [np.mean(ll) for ll in list_aucs1]
                    error = [np.std(ll) for ll in list_aucs1]

                    x_pos = np.arange(0,2*len(meth_val),2)
                
                    colors3 = list(np.repeat(['Maroon'],len(meth)))
                    s2 = 16
                    sc = 2

                    bars = ax.bar(x_pos, meth_val, yerr=error, alpha=1, edgecolor='k', align = 'center', capsize=sc, color = colors3)

                    ax.set_xticks(x_pos)
                    ax.set_ylabel('AUC', fontsize = s2)
                    ax.tick_params(axis='y', which='major', labelsize = s2)
                    ax.set_xticklabels(meth, rotation = 30, fontsize  = s2-3, ha='right', rotation_mode = 'anchor')
                    ax.set_title('Performance at discriminating pos-neg for peptide ' + pep + str(np.mean(list_auc_backrbm)), fontsize = s2)
                    ax.set_ylim([0,1]);
                    
                    ## print final table ##
                    if print_table:
                        list_aucs = list_aucs1
                        list_aucs_names = list_aucs_names1 
                        dataf = {list_aucs_names[k]:list_aucs[k] for k in range(len(list_aucs))}
                        df = pd.DataFrame(dataf, columns = list_aucs_names)
                        df.to_csv(name_folder + '/'+ add_str + 'final_aucs' + add_vj  + '.tsv', sep='\t', index=False)

Here we calculate the diffRBM units' single-site factors and test their performance at predicting contacts between the CDR3beta and the peptide

In [ ]:
makeVJ=0 ## for contacts, we need only the amino-acid part 

if makeVJ:
    strVJ = 'withVJ'
else:
    strVJ = 'withoutVJ'

print_table = 0

listP = [['YLQPRTFLL','NLVPMVATV','GLCTLVAML','GILGFVFTL'], ['YLQPRTFLL'], ['NLVPMVATV'], ['GLCTLVAML'],['GILGFVFTL']]

Mb = 1000000
l2f = 1/Mb
for RP in [1,0]:
    for Pi in range(len(listP)):
        list_pep=listP[Pi]
        list_ppv_pwm=[]
        list_ppv_rbm=[]
        npw=250

        ## min dist at which I see at least 1 contact per peptide -> 3.5
        for TH in [3.8,4,4.5,5]:
            name_folderF = rootf + '/Immunogenicity_model/PDB_structures/'
            perfs=[]

            range_len = [8,9,10,11]

            range_len_target  = [9]

            ## parameters for the alignment ##
            SA = 9
            SAmin = 8
            SAmax = 11

            makeextractionB = 0
            makeRW = 0
            vv = 0

            ## percentage to use in the training dataset ##
            B = 100 ## percentage for positives ##
            Bm = 100 ## percentage for negatives ##
            BB = 100 ## percentage for background ##

            ## Background model parameters ##
            l1bB = 0.001
            n_hB = 10
            listregB = [l1bB] ## lists of weight regularization in top RBM
            listhuB = [n_hB] 

            ## parameters of diffRBM ##
            listreg = [0.01] ## lists of weight regularization in top RBM
            listhu = [10] ## lists of hidden units in top RBM
            n_min = 300 ## maximal fields regularization

            ## RBM parameters ##
            ZF = False ## control the introduction of fields ##
            BN = True ## control batch_norm ##

            visible = 'Potts' # Nature of visible units potential. Here, Potts states.
            hidden = 'dReLU' # Nature of hidden units potential. Here, dReLU potential.
            seed = 0
            decay_after = 0.5 # Decay learning rate after 50% of iterations (default: 0.5). Value for RBM shown in paper: 0.5
            N_MC = 15
            fac = 1

            maketrainingB = 0 ## retrain the background data ##
            maketraining = 0 ## train diffRBM ##
            first_run = 0
            makefig = 0
            make_neg = 0
            maketrainingN = 0
            what_l2 = 'Standard'
            reweight_ppv = RP ## wheher one should re-weight same peptide entries 
            th = 0.1 ## similarity threshold if you set the reweighting: 0.1 (only same); 0.2 (1 mut away)
            if reweight_ppv==0:
                th=0
            if len(list_pep)>1:
                pepN='All'
            else:
                pepN=list_pep[0]
            make_unique_cont = 1 ## only unique seqs + set of contact positions
            make_unique = 0  ## only unique seqs

            l1=[]
            l1lin=[]
            l1b=[]
            l12=[]
            l1b1=[]
            l1b2=[]
            posR=[]
            posF=[]
            posB=[]
            posB1=[]
            posB2=[]
            l2=[]
            posRlin=[]
            posP=[]
            ligands=[]
            true_contacts_hla=[]
            rws_hla=[]

            for p0 in range(len(list_pep)):

                lik_pos=[]
                lik_neg=[]
                lik_back=[]
                lik_pos_pwm=[]
                lik_neg_pwm=[]
                lik_back_pwm=[]

                count=0
                list_pos=[]

                pep = list_pep[p0]


                file = pd.read_csv(name_folderF + '/contacts_ang' + str(TH) + '_fin.tsv', sep='\t')
                filea02 = file[file['Ligand'] == pep]


                ## CONTACT analysis: frequency of contact residues in structural data #                                   
                seqs0f = list(filea02['CDR3 seq'])
                cont0 = list(filea02['Pep contact positions (CDR3beta)'].values)
                seqs0 = list(filea02['CDR3 seq al'].values)

                # load peptide to be excluded in the training set
                seq_to_be_excluded = np.loadtxt(rootf + '/TCR_specificity_model/exclude_'+pep+'.txt', dtype=str)
                # prepare train_2num_withVJ
                vdgdb_df = pd.read_csv(rootf + '/TCR_specificity_model/diffRBM_'+pep+'/VDJdb_'+pep+'_WithAligned20.csv')
                vdgdb_df = vdgdb_df.drop_duplicates().reset_index(drop=True)
                train_data = vdgdb_df[~vdgdb_df["CDR3_beta"].isin(seq_to_be_excluded)]

                # do num2seq
                t_trainseq = train_data["ali_seq"].to_list()
                trainseq_2num = np.array(Proteins_utils.seq2num(t_trainseq), dtype=np.int16)
                # add to gen_seqs2num two integers for each row, obtained from V and J
                Vlist = train_data["TRBV_gene"].to_list()
                Jlist = train_data["TRBJ_gene"].to_list()

                t_df = pd.read_csv(rootf + '/TCR_specificity_model/diffRBM_'+pep+'/J2num_dict_train_train.csv')
                J2num_dict = dict(zip(t_df['J_gene'], t_df['RBM_VU_id']))
                t_df = pd.read_csv(rootf + '/TCR_specificity_model/diffRBM_'+pep+'/V2num_dict_train_train.csv')
                V2num_dict = dict(zip(t_df['V_gene'], t_df['RBM_VU_id']))

                train_2num_withVJ = add_VJ_info_2num(trainseq_2num, Vlist, Jlist, V2num_dict, J2num_dict)

                print(train_2num_withVJ.shape)

                if makeVJ:
                    train_data = np.copy(train_2num_withVJ)
                else:
                    train_data = np.copy(train_2num_withVJ[:,:-2])

                
                filename_cdr3 = rootf + '/TCR_specificity_model/emerson_training_test/train_data_0_aligned_20.txt' 
                filename_cdr3raw = rootf + '/TCR_specificity_model/emerson_training_test/train_data_0.txt' 

                train_seq0=[]
                with open(filename_cdr3) as f:
                    for line in f:
                        linesplit = line.strip().split('\n')
                        train_seq0.append(linesplit[0])

                train_dataB_num = convert_number(train_seq0)

                if makeVJ:
                    ffraw = pd.read_csv(filename_cdr3raw,sep='\t',header=None)
                    VlistB = list(np.array(ffraw)[:,1])
                    JlistB = list(np.array(ffraw)[:,2])
                    train_2num_withVJ_B = add_VJ_info_2num(train_dataB_num, VlistB, JlistB, V2num_dict, J2num_dict)
                    train_dataB = train_2num_withVJ_B[:Mb]
                else:
                    train_dataB = train_dataB_num[:Mb]

                if makeVJ:
                    n_cv = np.max(np.unique(train_dataB[:,-2])) + 1
                else:
                    n_cv=21

                Mb=len(train_dataB)
                pre_fields = utilities.average(train_dataB, c=n_cv)
                fields_back, pwmB = regularized_pwm_torch(torch.from_numpy(pre_fields), 1/(fac*Mb), n_iter=npw)
                if makefig:
                    sequence_logo.Sequence_logo(utilities.average(train_data,c=n_cv))

                decay_after = 0.5
                N_MC = 10

                batch_size = 2000
                n_iter = 40

                replB=0
                name_back = 'backRBM-100hu-lowreg-'+ strVJ +'-train_train_WithoutDuplicates_repl'+str(replB)+'_' + str(Mb)
                nameB = rootf + '/TCR_specificity_model/' +name_back +'.data'

                mybackRBM = rbm.RBM(n_h = 100, n_v = len(train_dataB[0]),n_cv= n_cv, visible='Potts', hidden='dReLU',random_state = seed, zero_field = False)
                if maketrainingB:
                    mybackRBM.fit(train_dataB, weights = None, batch_size = batch_size, n_iter = n_iter, l1b = 0.001, l2_fields = l2f, N_MC = N_MC, decay_after = decay_after, verbose = 1, shuffle_data=False, CD=False)
                    RBM_utils.saveRBM(nameB, mybackRBM)
                nameB = rootf + '/TCR_specificity_model/' +name_back +'.data'

                mybackRBM = RBM_utils.loadRBM(nameB)

                Mt = len(train_data)
                emp_freqsF = utilities.average(train_data, c=n_cv, weights = None)
                fields_temp, pwmP = regularized_pwm_torch(torch.from_numpy(emp_freqsF), 1/(fac*Mt), n_iter= npw)

                cont=[]
                seqs=[]
                for rl in range(len(cont0)):
                    if type(cont0[rl])!=float:
                        cont.append([int(r) for r in cont0[rl].split(' ')])                       
                        seqs.append(seqs0[rl])

                if make_unique_cont:
                    seqs_u2=[]
                    for N in range(len(seqs)):
                        list2=''
                        for c in cont[N]:
                            list2 += str(c)
                        seqs_u2.append(seqs[N] + list2)

                    seqs_u3 = list(np.unique(seqs_u2))
                    inds_u = [seqs_u2.index(s) for s in seqs_u3]

                    seqs=[seqs[i] for i in inds_u]
                    cont=[cont[i] for i in inds_u]

                if make_unique:
                    seqs_u = []
                    count_u = []
                    for ss in range(len(seqs)):
                        if seqs[ss] not in seqs_u:
                            seqs_u.append(seqs[ss])
                            if seqs.count(seqs[ss])>1:
                                hits = [i for i in range(len(seqs)) if seqs[i] == seqs[ss]]
                                first=cont[ss]
                                for y in hits:
                                    ov = overlap_seqs(first,cont[y])
                                    first = list(ov)
                                count_u.append(ov)
                            else:
                                count_u.append(cont[ss])        
                    seqs=seqs_u
                    cont=count_u

                seqs_n=convert_number(seqs)


                learning_rate = None # default behaviour
                decay_after = 0.5
                N_MC = 10

                if pep == 'YLQPRTFLL':
                    bs = 16
                else:
                    bs = 100
                if pep == 'NLVPMVATV': 
                    bs = 200
                batch_size = bs

                n_iter = int(2e4) // (train_data.shape[0] // bs)

                n_h = 20
                lib = 0.01

                l2f_top= 1/Mt
                l2f= 1/Mt

                out_par0 = '_RW' + str(makeRW) + '_TR' + str(B) 
                out_par = '_nh' + str(n_h) + '_l12' + str(lib) + '_ZF' + str(ZF) + out_par0
                name_top_lin = rootf + '/TCR_specificity_model/diffRBM_'+pep+'/models/topmodellin_test_contacts' + out_par + '.data'

                postRBM = rbm.RBM(n_h = mybackRBM.n_h + n_h, n_v = mybackRBM.n_v, n_cv= mybackRBM.n_cv, visible='Potts', hidden='dReLU')
                diffRBM = diffrbm.DiffRBM(mybackRBM, postRBM)
                diffRBM.update_post_from_back(vlayer=True, hlayer=True)

                if maketraining:
                    diffRBM.fit_top(train_data, batch_size = batch_size, n_iter = n_iter, l1b = lib, l2_fields = l2f, l2_fields_top = 0, N_MC = N_MC, decay_after = decay_after, verbose = vv, vverbose = vv, batch_norm=True,callback = produce_callback_NOweights(diffRBM)) 
                    RBM_utils.saveRBM(name_top_lin, diffRBM)
                    dRBM_lin = diffRBM

                dRBM_lin = RBM_utils.loadRBM(name_top_lin)
                RBM_back = dRBM_lin.RBMback
                g_top_lin = dRBM_lin.top_rbm().vlayer.fields

                l2f_top = 1/Mt
                l2f= 1/Mt
                postRBM = rbm.RBM(n_h = mybackRBM.n_h + n_h, n_v = mybackRBM.n_v, n_cv= mybackRBM.n_cv, visible='Potts', hidden='dReLU')
                diffRBM = diffrbm.DiffRBM(mybackRBM, postRBM)
                diffRBM.update_post_from_back(vlayer=True, hlayer=True)

                name_top = rootf + '/TCR_specificity_model/diffRBM_'+pep+'/models/topmodel_test_contacts' + out_par + '.data'

                if maketraining:
                    diffRBM.fit_top(train_data, batch_size = batch_size, n_iter = n_iter, l1b = lib, l2_fields = l2f, l2_fields_top = 0, N_MC = N_MC, decay_after = decay_after, verbose = vv, vverbose = vv, batch_norm=True)
                    RBM_utils.saveRBM(name_top, diffRBM)
                    dRBM = diffRBM
                dRBM = RBM_utils.loadRBM(name_top)

                RBM_back = dRBM.RBMback
                g_back = RBM_back.vlayer.fields
                g_top = dRBM.top_rbm().vlayer.fields
                g_post = dRBM.RBMpost.vlayer.fields
                RBMq = dRBM.top_rbm()

                SA=20
                for N in range(len(seqs_n)):

                    l10lin = [g_top_lin[i, seqs_n[N][i]] for i in range(SA) if seqs_n[N][i]!=20]

                    hm = RBMq.hlayer.mean_from_inputs(RBMq.vlayer.compute_output(seqs_n[N], RBMq.weights, direction ='up'), beta = 1)
                    I_top = RBMq.hlayer.compute_output(hm, RBMq.weights, direction='down')
                    l10 = [g_top[i, seqs_n[N][i]] + sum(I_top[:,i, seqs_n[N][i]]) for i in range(SA) if seqs_n[N][i]!=20]

                    hm = RBMq.hlayer.sample_from_inputs(RBMq.vlayer.compute_output(seqs_n[N], RBMq.weights, direction ='up') , beta = 1)

                    I_top2 = RBMq.hlayer.compute_output(hm, RBMq.weights, direction='down')
                    l1020 = site_conservation_scores(RBMq, seqs_n[N], A=21, L=len(seqs_n[N]))
                    l102= [l1020[l] for l in range(len(l1020)) if seqs_n[N][l]!=20]

                    l1.append(l10)
                    l12.append(l102)
                    l1lin.append(l10lin)

                    posR.append(list(np.argsort(l10)[::-1] + 1))
                    posRlin.append(list(np.argsort(l10lin)[::-1] + 1))
                    posF.append(list(np.argsort(l102)[::-1] + 1))

                    l1b0 = [fields_temp[i, seqs_n[N][i]] - fields_back[i, seqs_n[N][i]] for i in range(SA) if seqs_n[N][i]!=20]
                    l1b.append(l1b0)

                    l1b01 = [pwmP[i, seqs_n[N][i]] for i in range(SA) if seqs_n[N][i]!=20]

                    l1b1.append(l1b01)

                    posB1.append(list(np.argsort(l1b01)[::-1] + 1))

                    posB.append(list(np.argsort(l1b0)[::-1] + 1))

                    count+=1

                ligands += list(seqs)
                true_contacts_hla += list(cont)
                if reweight_ppv:
                    rws0 = utilities_diffrbm.compute_MSA_weights(seqs_n, threshold = th)
                    rws_hla += list(rws0)

            Leff = len(ligands)
            if reweight_ppv:
                Leff = sum(rws_hla)

            SAl = np.min([len(posR[g]) for g in range(len(posR))])

            s2=16
            pos=np.copy(posR)
            ppvR=[]
            lm=[]
            for u in range(SAl):
                predP=0
                for i in range(len(ligands)):
                    fac=1
                    if reweight_ppv:
                        fac=rws_hla[i]
                    ov = overlap_seqs(pos[i][:u+1], true_contacts_hla[i])
                    if u==0 and len(ov)==0:
                        lm.append(ligands[i])
                    predP += fac*len(ov)/len(true_contacts_hla[i][:u+1])
                ppvR.append(predP/Leff)

            pos=np.copy(posRlin)
            ppvRlin=[]
            lm=[]
            for u in range(SAl):
                predP=0
                for i in range(len(ligands)):
                    fac=1
                    if reweight_ppv:
                        fac=rws_hla[i]
                    ov = overlap_seqs(pos[i][:u+1], true_contacts_hla[i])
                    if u==0 and len(ov)==0:
                        lm.append(ligands[i])
                    predP += fac*len(ov)/len(true_contacts_hla[i][:u+1])
                ppvRlin.append(predP/Leff)

            pos = np.copy(posB)
            ppvB=[]
            for u in range(SAl):
                predP=0
                for i in range(len(ligands)):
                    fac=1
                    if reweight_ppv:
                        fac=rws_hla[i]
                    ov = overlap_seqs(pos[i][:u+1], true_contacts_hla[i])
                    predP += fac*len(ov)/len(true_contacts_hla[i][:u+1])
                ppvB.append(predP/Leff)

            pos = np.copy(posB1)
            ppvB1=[]
            for u in range(SAl):
                predP=0
                for i in range(len(ligands)):
                    fac=1
                    if reweight_ppv:
                        fac=rws_hla[i]
                    ov = overlap_seqs(pos[i][:u+1], true_contacts_hla[i])
                    predP += fac*len(ov)/len(true_contacts_hla[i][:u+1])
                ppvB1.append(predP/Leff)


            pos = np.copy(posF)
            ppvF=[]
            for u in range(SAl):
                predP=0
                for i in range(len(ligands)):
                    fac=1
                    if reweight_ppv:
                        fac=rws_hla[i]
                    ov = overlap_seqs(pos[i][:u+1], true_contacts_hla[i])
                    predP += fac*len(ov)/len(true_contacts_hla[i][:u+1])
                ppvF.append(predP/Leff)

            list_ppv_pwm.append(ppvB)
            list_ppv_rbm.append(ppvR)

            ppv_random_all=[]
            for f in range(1000):
                pos_random=[]
                for u in range(len(ligands)):
                    seq_random=list(np.arange(len([t for t in ligands[u] if t != '-']))+1)
                    random.shuffle(seq_random)
                    pos_random.append(seq_random)
                pos = np.copy(pos_random)
                ppv_random=[]
                lm=[]
                for u in range(SAl):
                    predP=0
                    for i in range(len(ligands)):
                        fac=1
                        if reweight_ppv:
                            fac=rws_hla[i]
                        ov = overlap_seqs(pos[i][:u+1], true_contacts_hla[i])

                        predP += fac*len(ov)/len(true_contacts_hla[i][:u+1])
                    ppv_random.append(predP/Leff)
                ppv_random_all.append(ppv_random)
            ppv_random = np.mean(ppv_random_all,axis=0)

            if print_table:
                datao = {
                'Random': ppv_random,
                'Frequency': ppvB1,
                'diffPWM': ppvB,
                'topRBM (lin)': ppvRlin,
                'topRBM2': ppvF,
                'topRBM1': ppvR
                }

                df = pd.DataFrame(datao, columns = ['Random','Frequency', 'diffPWM', 'topRBM (lin)', 'topRBM2', 'topRBM1'])
                df.to_csv(rootf + '/TCR_specificity_model/Results/prediction_contacts_ang' + str(TH) + '_pep' + pepN + '_rew' + str(th) + '.tsv', sep='\t')

            cont = true_contacts_hla;
            cont_R=[]
            for c in range(len(cont)):
                cont_R.append(list(np.array(cont[c])-len(l1[c])-1))

            freqs_R=[]
            sym_R=[]
            for u in range(-SA,0):
                cou=0
                sym_R.append(u)
                for cc in range(len(cont_R)):
                    fac=1
                    if reweight_ppv:
                        fac = rws_hla[cc]
                    cou+=fac*cont_R[cc].count(u+1)
                freqs_R.append(cou/Leff)

            freqs=[]
            sym=[]
            for u in range(SA):
                cou=0
                sym.append(u+1)
                for cc in range(len(cont)):
                    fac=1
                    if reweight_ppv:
                        fac = rws_hla[cc]
                    cou+=fac*cont[cc].count(u+1)
                freqs.append(cou/Leff)

            l1ave_R=[]
            l1bave_R=[]
            for j in range(-SA,0):
                l1_temp = [l1[l] for l in range(len(l1)) if j + len(l1[l]) + 1 > 0 and len(l1[l]) > j + len(l1[l]) + 1 ]
                l1b_temp = [l1b[l] for l in range(len(l1b)) if j + len(l1[l]) + 1 > 0 and len(l1b[l]) > j + len(l1[l]) + 1]
                if len(l1_temp):
                    l1ave_R.append(sum([l[j + len(l) + 1] for l in l1_temp])/len(l1_temp))
                    l1bave_R.append(sum([l[j + len(l) + 1] for l in l1b_temp])/len(l1b_temp))

            l1ave=[]
            l1bave=[]
            for i in range(SA):
                l1_temp = [l1[l] for l in range(len(l1)) if len(l1[l]) > i]
                l1b_temp = [l1b[l] for l in range(len(l1b)) if len(l1b[l]) > i]
                if len(l1_temp):
                    l1ave.append(sum([l[i] for l in l1_temp])/len(l1_temp))
                    l1bave.append(sum([l[i] for l in l1b_temp])/len(l1b_temp))



            if print_table and TH==4 and Pi==0 and RP==1:
                datao = {
                'sym': sym[:len(l1ave)],
                'freqs': freqs[:len(l1ave)],
                'pred_diffpwm': l1bave,
                'pred_rbm': l1ave
                }
                df = pd.DataFrame(datao, columns = ['sym', 'freqs', 'pred_diffpwm', 'pred_rbm'])
                df.to_csv(rootf + '/TCR_specificity_model/Results/frequency_contacts_ang' + str(TH) + '_rew' + str(th) + '.tsv', sep='\t')
                datao = {
                'sym_R': sym_R[-len(l1ave_R):],
                'freqs_R': freqs_R[-len(l1ave_R):],
                'pred_diffpwm_R': l1bave_R,
                'pred_rbm_R': l1ave_R
                }
                df = pd.DataFrame(datao, columns = ['sym_R', 'freqs_R', 'pred_diffpwm_R', 'pred_rbm_R'])
                df.to_csv(rootf + '/TCR_specificity_model/Results/frequency_contacts_ang' + str(TH) + '_rew' + str(th) + '_R.tsv', sep='\t')